In [ ]:
import datetime
from getpass import getpass
from typing import List, Optional

import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from huggingface_hub import HfApi
from huggingface_hub.hf_api import HfApi, HfFolder
from pydantic import BaseModel, IPvAnyAddress, validator
from requests.exceptions import HTTPError

In [ ]:
ENDPOINT_WHITELIST = "https://collaborative-training-auth.huggingface.co"


class ANSI:
    """
    Helper for en.wikipedia.org/wiki/ANSI_escape_code
    """

    _bold = "\u001b[1m"
    _red = "\u001b[31m"
    _gray = "\u001b[90m"
    _reset = "\u001b[0m"

    @classmethod
    def bold(cls, s):
        return "{}{}{}".format(cls._bold, s, cls._reset)

    @classmethod
    def red(cls, s):
        return "{}{}{}".format(cls._bold + cls._red, s, cls._reset)

    @classmethod
    def gray(cls, s):
        return "{}{}{}".format(cls._gray, s, cls._reset)


class HFTokenManagemment:
    def __init__(self):
        self._api = HfApi()

    def login(self):
        print(  # docstyle-ignore
            """
        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
        """
        )
        username = input("Username: ")
        password = getpass()
        try:
            token = self._api.login(username, password)
        except HTTPError as e:
            # probably invalid credentials, display error message.
            print(e)
            print(ANSI.red(e.response.text))
            exit(1)
        HfFolder.save_token(token)
        print("Login successful")
        print("Your token:", token, "\n")
        print("Your token has been saved to", HfFolder.path_token)

    def logout(self):
        token = HfFolder.get_token()
        if token is None:
            print("Not logged in")
            exit()
        HfFolder.delete_token()
        self._api.logout(token)
        print("Successfully logged out.")

In [ ]:
# Define models to help request to API
class ExperimentJoinInput(BaseModel):
    """
    All common characteristics of our Experiment resource
    """

    peer_public_key: Optional[bytes]  # bytes


class HivemindAccess(BaseModel):
    username: str
    peer_public_key: bytes
    expiration_time: datetime.datetime
    signature: bytes


class ExperimentJoinOutput(BaseModel):
    """
    All common characteristics of our Experiment resource
    """

    coordinator_ip: Optional[IPvAnyAddress]
    coordinator_port: Optional[int]
    hivemind_access: HivemindAccess
    auth_server_public_key: bytes

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port

In [ ]:
# As a HF user, you ask a HF token session
token_management = HFTokenManagemment()
token_management.login()

# Ask to join an experiment

In [ ]:
# This is the content request for the new experiment to create

private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
serialized_public_key = private_key.public_key().public_bytes(
    encoding=serialization.Encoding.OpenSSH, format=serialization.PublicFormat.OpenSSH
)
experiment_join_input = ExperimentJoinInput(peer_public_key=serialized_public_key)

values = experiment_join_input.dict()
# Make the values JSON serializable
values["peer_public_key"] = values["peer_public_key"].decode("utf-8")

id_experiment = 1  # Change me

In [ ]:
# Send request to server
path = f"{ENDPOINT_WHITELIST}/api/experiments/join/{id_experiment}/"

headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
r = requests.put(path, headers=headers, json={"experiment_join_input": values})
r.raise_for_status()
d = r.json()

In [ ]:
# Show server response
d

In [ ]:
ouput = ExperimentJoinOutput(**d)
ouput

In [ ]:
# As a HF user, you terminate your session (your token is not longer valid)
token_management.logout()